In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import io
import osmnx as ox
from shapely.geometry import Polygon, LineString, Point
import os
import sys
sys.path.append(os.path.abspath(".."))
from src.gvi_funcs import get_responses
from src.gvi_funcs import greenviewindex

#### create a list of points for API calls

In [ ]:
api_key = os.environ.get('api_key')

In [ ]:
polygon = [[ 13.335820607449087, 52.479180814784826 ], [ 13.319144034173492, 52.481940744889087 ], [ 13.319144034173492, 52.481940744889087 ], [ 13.303192529301191, 52.491654321569996 ], [ 13.287966092832171, 52.502689873200431 ], [ 13.293041571655179, 52.511516320373843 ], [ 13.288691161235455, 52.519899803498745 ], [ 13.29014129804203, 52.526958338172669 ], [ 13.301017324091331, 52.528281687156365 ], [ 13.324219512996502, 52.530046090443768 ], [ 13.353222249127967, 52.534015738673432 ], [ 13.36337320677398, 52.538426038072501 ], [ 13.380774848452859, 52.545481595702277 ], [ 13.398176490131736, 52.54636346067177 ], [ 13.411227721390897, 52.546804386512193 ], [ 13.42355388424677, 52.541953958639048 ], [ 13.432979773489496, 52.539749041508585 ], [ 13.439505389119075, 52.537985028067389 ], [ 13.44675607315194, 52.534456788548354 ], [ 13.45183155197495, 52.526958338172669 ], [ 13.456907030797952, 52.524311520584895 ], [ 13.462707578024245, 52.523208632849773 ], [ 13.469958262057109, 52.518796804998914 ], [ 13.473583604073546, 52.513943282616047 ], [ 13.473583604073546, 52.509309874748915 ], [ 13.470683330460398, 52.505117322867868 ], [ 13.466695454242322, 52.500703678312291 ], [ 13.464882783234104, 52.497834571715572 ], [ 13.461257441217672, 52.494744555219448 ], [ 13.45799463340288, 52.492316532775945 ], [ 13.457632099201236, 52.489115752903984 ], [ 13.457088297898771, 52.486797801367324 ], [ 13.457088297898771, 52.484038175975996 ], [ 13.456000695293843, 52.4796224152363 ], [ 13.454913092688912, 52.473770849536344 ], [ 13.452375353277407, 52.472114604750061 ], [ 13.444218333740436, 52.470789564040665 ], [ 13.439505389119072, 52.469685332974741 ], [ 13.432073437985386, 52.468470646803397 ], [ 13.426816692061557, 52.46791850564356 ], [ 13.420834877734441, 52.467145496383672 ], [ 13.415396864709793, 52.466262040606331 ], [ 13.408508714878566, 52.465489002255779 ], [ 13.402889434753099, 52.465709871741218 ], [ 13.39781395593009, 52.467366357557452 ], [ 13.394732415216122, 52.468912354744376 ], [ 13.389294402191474, 52.470789564040665 ], [ 13.383131320763537, 52.471728138664126 ], [ 13.378055841940533, 52.472335440989724 ], [ 13.374067965722457, 52.472280232033704 ], [ 13.370533257256433, 52.472280232033704 ], [ 13.368720586248219, 52.472997943059788 ], [ 13.367361082992057, 52.473770849536344 ], [ 13.366545381038357, 52.475427031989611 ], [ 13.365185877782197, 52.477248760694756 ], [ 13.363645107425214, 52.478352801991761 ], [ 13.361379268664942, 52.47956721542225 ], [ 13.35893216280385, 52.480340006516634 ], [ 13.355034920136184, 52.480395205361077 ], [ 13.351500211670158, 52.479953612666179 ], [ 13.348781205157836, 52.478960012896962 ], [ 13.346062198645511, 52.478628807987505 ], [ 13.343886993435651, 52.479015213472806 ], [ 13.341258620473734, 52.479346415473572 ], [ 13.339174048814286, 52.479291215313218 ], [ 13.335820607449087, 52.479180814784826 ]]

In [ ]:
polygon = Polygon(polygon)

In [ ]:
G = ox.graph_from_polygon(polygon=polygon, simplify=True, network_type='drive')

In [ ]:
ox.plot_graph(G)

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)

In [ ]:
edges.reset_index(drop=True, inplace=True)

In [ ]:
df = edges
df.reset_index(drop=True, inplace=True)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs('EPSG:4326', inplace=True)
gdf.head(3)

In [ ]:
distance_delta = 0.001

In [ ]:
#create points in equal steps, defined by distance_delta
point_list = []

for line in df['geometry']:
    distances = np.arange(0, line.length, distance_delta)
    points = [line.interpolate(distance) for distance in distances]
    point_list.extend(points)

In [ ]:
# creatin a unique list of points --> multiple linestrings start at same point
point_list = list(dict.fromkeys(point_list))

### get and safe image for every point in point_list

In [ ]:
lon_list = [point.x for point in point_list]
lat_list = [point.y for point in point_list]

In [ ]:
request_list = [f"{lat},{lon}" for lat, lon in zip(lat_list, lon_list)]

In [ ]:
request_list = request_list[:5]
point_list = point_list[:5]

In [ ]:
data = [get_responses(latlon, point, api_key) for latlon, point in zip(request_list, point_list)]

In [ ]:
df = pd.DataFrame(data)

In [ ]:
cols = ['Image', 'Date', 'Point']
df.columns = cols

In [ ]:
image_dir = '../data/images_xx'
os.makedirs(image_dir, exist_ok=True)

metadata = []

for idx, row in df.iterrows():
    filename = f'image_{idx}.jpg'  
    file_path = os.path.join(image_dir, filename)

    img = row['Image']
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    img.save(file_path, 'JPEG')
    metadata.append({
        'Filename': filename,
        'Date': row['Date'],
        'Point': row['Point'],
        'OriginalMode': row['Image'].mode 
    })

metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv('../data/image_metadata_xx.csv', index=False)